In [22]:
import torch
params = torch.load("./model_weights.pt",map_location="cpu")
params.keys()

odict_keys(['conv1.weight', 'conv1.bias', 'conv1.scale', 'conv1.zero_point', 'conv2.weight', 'conv2.bias', 'conv2.scale', 'conv2.zero_point', 'fc1.scale', 'fc1.zero_point', 'fc1._packed_params.dtype', 'fc1._packed_params._packed_params', 'fc2.scale', 'fc2.zero_point', 'fc2._packed_params.dtype', 'fc2._packed_params._packed_params', 'fc3.scale', 'fc3.zero_point', 'fc3._packed_params.dtype', 'fc3._packed_params._packed_params', 'quant.scale', 'quant.zero_point'])

#Weights for Stage 1

In [23]:
weights = params["conv1.weight"].int_repr()
num_kern = weights.shape[0]
channels = weights.shape[1]
groups = (channels//8)

rows = weights.shape[2]
cols = weights.shape[3]

if channels%8 != 0:
  groups+=1

rem_channels = (8*groups) - channels

with open("weights1.txt", "w") as w_file:
  for ker_no in range(num_kern):
    for row in range(rows):
      for col in range(cols):

        for chn in range(channels):
          w_file.write("{}\n".format(weights[ker_no][chn][row][col]))
        for rem_chn in range(rem_channels):
          w_file.write("0\n")





#Weights for Stage 2

In [24]:
weights = params["conv2.weight"].int_repr()
num_kern = weights.shape[0]
channels = weights.shape[1]
groups = (channels//8)

rows = weights.shape[2]
cols = weights.shape[3]

if channels%8 != 0:
  groups+=1

rem_channels = (8*groups) - channels

with open("weights2.txt", "w") as w_file:
  for ker_no in range(num_kern):
    for row in range(rows):
      for col in range(cols):

        for chn in range(channels):
          w_file.write("{}\n".format(weights[ker_no][chn][row][col]))
        for rem_chn in range(rem_channels):
          w_file.write("0\n")





# Weights for Stage 3

In [25]:
weights = params["fc1._packed_params._packed_params"][0].int_repr()
num_kern = weights.shape[0]
channels = weights.shape[1]
groups = (channels//8)


if channels%8 != 0:
  groups+=1
print(groups)
rem_channels = (8*groups) - channels
print(rem_channels)
# weights.shape
with open("weights3.txt", "w") as w_file:
  for ker_no in range(num_kern):
    for chn in range(channels):
      w_file.write("{}\n".format(weights[ker_no][chn]))
    for rem_chn in range(rem_channels):
      w_file.write("0\n")





32
0


# Weights for Stage 4

In [26]:
weights = params["fc2._packed_params._packed_params"][0].int_repr()
num_kern = weights.shape[0]
channels = weights.shape[1]
groups = (channels//8)


if channels%8 != 0:
  groups+=1

print(channels, " ",groups)
rem_channels = (8*groups) - channels
# weights.shape
with open("weights4.txt", "w") as w_file:
  for ker_no in range(num_kern):
    for chn in range(channels):
      w_file.write("{}\n".format(weights[ker_no][chn]))
    for rem_chn in range(rem_channels):
      w_file.write("0\n")





120   15


In [27]:
print(weights)

tensor([[  0, -54, -52,  ...,  -6, -34,  -1],
        [ -6, -34, -26,  ..., -11, -11,  -8],
        [ 22, -46, -60,  ...,  10, -25,   4],
        ...,
        [ 15, -30, -33,  ..., -13, -19,  -6],
        [ 18, -23,   2,  ..., -28, -18,   5],
        [  2, -30, -14,  ..., -15, -38,   9]], dtype=torch.int8)


#Weights for Stage 5


In [28]:
weights = params["fc3._packed_params._packed_params"][0].int_repr()
num_kern = weights.shape[0]
channels = weights.shape[1]
groups = (channels//8)


if channels%8 != 0:
  groups+=1

print(channels, " ",groups)
rem_channels = (8*groups) - channels
# weights.shape
with open("weights5.txt", "w") as w_file:
  for ker_no in range(num_kern):
    for chn in range(channels):
      w_file.write("{}\n".format(weights[ker_no][chn]))
    for rem_chn in range(rem_channels):
      w_file.write("0\n")





84   11


# Utils Data like scale and zero point

In [29]:
weights = params["fc3._packed_params._packed_params"][0].int_repr()
num_kern = weights.shape[0]
channels = weights.shape[1]
groups = (channels//8)


if channels%8 != 0:
  groups+=1

print(channels, " ",groups)
rem_channels = (8*groups) - channels
# weights.shape
with open("utils.txt", "w") as w_file:
  # For Stage 1
  inp_scale = params["quant.scale"].item()
  inp_zero_point = params["quant.zero_point"].item()
  ker_scale = params["conv1.weight"].q_scale()
  ker_zero_point = params["conv1.weight"].q_zero_point()
  conv_scale = params["conv1.scale"].item()
  conv_zero_point = params["conv1.zero_point"].item()

  w_file.write("{}\n{}\n".format(inp_scale,inp_zero_point))
  w_file.write("{}\n{}\n".format(ker_scale,ker_zero_point))
  w_file.write("{}\n{}\n".format(conv_scale,conv_zero_point))

  # For Stage 2
  inp_scale = params["conv1.scale"].item()
  inp_zero_point = params["conv1.zero_point"].item()
  ker_scale = params["conv2.weight"].q_scale()
  ker_zero_point = params["conv2.weight"].q_zero_point()
  conv_scale = params["conv2.scale"].item()
  conv_zero_point = params["conv2.zero_point"].item()

  w_file.write("{}\n{}\n".format(inp_scale,inp_zero_point))
  w_file.write("{}\n{}\n".format(ker_scale,ker_zero_point))
  w_file.write("{}\n{}\n".format(conv_scale,conv_zero_point))

  # For Stage 3
  inp_scale = params["conv2.scale"].item()
  inp_zero_point = params["conv2.zero_point"].item()
  ker_scale = params["fc1._packed_params._packed_params"][0].q_scale()
  ker_zero_point = params["fc1._packed_params._packed_params"][0].q_zero_point()
  conv_scale = params["fc1.scale"].item()
  conv_zero_point = params["fc1.zero_point"].item()

  w_file.write("{}\n{}\n".format(inp_scale,inp_zero_point))
  w_file.write("{}\n{}\n".format(ker_scale,ker_zero_point))
  w_file.write("{}\n{}\n".format(conv_scale,conv_zero_point))

  # For Stage 4
  inp_scale = params["fc1.scale"].item()
  inp_zero_point = params["fc1.zero_point"].item()
  ker_scale = params["fc2._packed_params._packed_params"][0].q_scale()
  ker_zero_point = params["fc2._packed_params._packed_params"][0].q_zero_point()
  conv_scale = params["fc2.scale"].item()
  conv_zero_point = params["fc2.zero_point"].item()

  w_file.write("{}\n{}\n".format(inp_scale,inp_zero_point))
  w_file.write("{}\n{}\n".format(ker_scale,ker_zero_point))
  w_file.write("{}\n{}\n".format(conv_scale,conv_zero_point))

  # For Stage 5
  inp_scale = params["fc2.scale"].item()
  inp_zero_point = params["fc2.zero_point"].item()
  ker_scale = params["fc3._packed_params._packed_params"][0].q_scale()
  ker_zero_point = params["fc3._packed_params._packed_params"][0].q_zero_point()
  conv_scale = params["fc3.scale"].item()
  conv_zero_point = params["fc3.zero_point"].item()

  w_file.write("{}\n{}\n".format(inp_scale,inp_zero_point))
  w_file.write("{}\n{}\n".format(ker_scale,ker_zero_point))
  w_file.write("{}\n{}\n".format(conv_scale,conv_zero_point))







84   11


In [30]:
type(params["quant.scale"].item())

float